**Experiment 2: linear regression and selecting features that occur a certain amount of time**

*Background*: Experiment 1 showed that some sort of feature selection is required, but also that just picking the k highest scoring features leads to overfitting.

*Goal*: Determine if it is possible to predict the year in which a text was written using regression.

*Strategies*:
- Train on features that occur a certain amount of time

*Relevance*:
- If this experiment works, it is possible to estimate years for corpora that have NA's in this variable.

*Success criteria*:
- Consistent findings over training-, test- and validation set
- predicted year is not more than ten years away from the true year

*Corpora*:
- DTA
- CLMET

*Result*: 
Only using features that occur in 800 out of 899 documents solved the problem of overfitting. 800 out of 899 is about 90% of the training set.

The linear regression seems to try to model a normal distribution, which does not reflect the real distribution of years in the DTA (cf analysis notebook). Therefore, it seems that linear regression is not flexible enough to model the data correctly.

*MSE DTA Train*: 2259.8

*MSE DTA Val*: 3202.51

*MSE DTA Test*: 4504.35

In [1]:
import pandas as pd
import numpy as np
import nltk

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest , f_regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import sklearn.utils
import re

import eli5

/Users/dianaenggist/opt/anaconda3/envs/R_python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/dianaenggist/opt/anaconda3/envs/R_python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Code example: https://stackoverflow.com/questions/39839112/the-easiest-way-for-getting-feature-names-after-running-selectkbest-in-scikit-le
def features_to_names(features, feature_names):
    features_selected = []

    for bool, feature in zip(features, feature_names):
        if bool:
            features_selected.append(feature)
    return features_selected

In [3]:
train_full = pd.read_csv('/Volumes/Korpora/Train/DTA_train_tokenized.csv', sep=';')
val_full = pd.read_csv('/Volumes/Korpora/Val/DTA_val_tokenized.csv', sep=';')
test_full = pd.read_csv('/Volumes/Korpora/Test/DTA_test_tokenized.csv', sep=';')

In [4]:
print('Length train set: ',len(train_full))
print('Length validation set: ', len(val_full))
print('Length test set: ', len(test_full))

Length train set:  899
Length validation set:  225
Length test set:  281


In [5]:
#build tokenizer that just substitutes '[' and ']' with ','
def tokenizer_word(doc):
    doc = re.sub('[(\[+)|(\]+)]', '', doc)
    doc = re.split(',', doc)
    return doc

In [6]:
#function for assembling predictions in order to find out how features are weighted

def collect_predictions(dataset, classifier,vectorizer, feature_names, pipeline):
    predictions = eli5.explain_weights_df(classifier,vec=vectorizer, feature_names=feature_names)
    
    predictions = predictions.drop(['target'], axis=1)
    
    
    predictions['YEAR'] = 0
    
    

    for instance in range (0, len(dataset)):
        pred = eli5.explain_prediction_df(classifier, dataset[instance], vec=vectorizer, feature_names=feature_names)
        source_text = pd.DataFrame([[dataset[instance]]])
        year_pred = pipeline.predict(source_text[0])
        pred['weight_value'] = pred['weight'] * pred['value']
        pred['instance'] = instance
        
        
        pred = pred.drop(['target','weight','value'], axis=1)
        
    
        pred['YEAR'] = np.round(year_pred[0])
    
        predictions = pd.concat([predictions, pred])
    
    
    
    return predictions

In [8]:
train_x = train_full['Text']
train_y = train_full['Publication_year']

val_x = val_full['Text']
val_y = val_full['Publication_year']

test_x = test_full['Text']
test_y = test_full['Publication_year']

CountVectorizer has an attribute called min_df that can be set to an integer or a float. If a word occurs less often than min_df (count or distribution), it is removed. I set min_df to 10 for the next experiment.

In Experiment 1, the model performed best on a set with 22000 features, so I start with that value.

In [ ]:
reg_1 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 10)),
                    ('feature_selector', SelectKBest(f_regression, k=22000)),
                         ('ridge_reg', linear_model.Ridge())
                        ])



In [ ]:
reg_1.fit(train_x, train_y)

In [ ]:
y_pred_train = reg_1.predict(train_x)
mean_squared_error(train_y, y_pred_train)

In [ ]:
y_pred_val = reg_1.predict(val_x)

mean_squared_error(val_y, y_pred_val)

In [ ]:
features = reg_1['feature_selector'].get_support()
feature_names = reg_1['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

expl = eli5.explain_weights_df(reg_1['ridge_reg'],vec=reg_1['unigram_vectorizer'],target_names=(train_y),feature_names=features_selected)




In [ ]:
print(expl.head(2))

It is still overfitting heavily, so I raise the threshold to 100.

Since there are less than 22k features over the threshold, I set k to 'all' for now, because else the feature selector does complain.

In [ ]:
reg_2 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 100)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [ ]:
reg_2.fit(train_x, train_y)

In [ ]:
y_pred_train = reg_2.predict(train_x)
mean_squared_error(train_y, y_pred_train)

In [ ]:
y_pred_val = reg_2.predict(val_x)

mean_squared_error(val_y, y_pred_val)

In [ ]:
features = reg_2['feature_selector'].get_support()
feature_names = reg_2['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.show_weights(reg_2['ridge_reg'],vec=reg_2['unigram_vectorizer'], feature_names=features_selected)

In [ ]:
reg_3 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 200)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [ ]:
reg_3.fit(train_x, train_y)

In [ ]:
y_pred_train = reg_3.predict(train_x)
mean_squared_error(train_y, y_pred_train)

In [ ]:
y_pred_val = reg_3.predict(val_x)

mean_squared_error(val_y, y_pred_val)

In [ ]:
features = reg_3['feature_selector'].get_support()
feature_names = reg_3['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.show_weights(reg_3['ridge_reg'],vec=reg_3['unigram_vectorizer'], feature_names=features_selected)

Since the model is still overfitting, I reverse the experiment, and I just pick features that occur in 800 out of 899 documents.

In [9]:
reg_4 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 800)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [10]:
reg_4.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('unigram_vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=800,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_word at 0x1d9359c20>,
                                 vocabulary=None)),
                ('feature_selector',
                 SelectKBest(k='all',
                             score_func=<function f_regression at 0x12071cdd0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr

In [11]:
y_pred_train = reg_4.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2259.798810884709

In [12]:
y_pred_val = reg_4.predict(val_x)

mean_squared_error(val_y, y_pred_val)

3202.5089784523752

In [13]:
y_pred_test = reg_4.predict(test_x)
mean_squared_error(test_y, y_pred_test)

4504.347276304357

In [14]:
features = reg_4['feature_selector'].get_support()
feature_names = reg_4['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.explain_weights(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

Weight?,Feature
+1769.725,<BIAS>
+0.282,'keine'
+0.261,'weiter'
+0.217,'ja'
+0.181,'that'
+0.178,'leicht'
+0.176,'vielen'
+0.155,'finden'
+0.153,'erhalten'
+0.152,'liegen'


In [15]:
len(features_selected)

215

The MSE of the train and val set converged now, so going after the stop words seems to be a pretty good idea. The model trained on 215 features.

In [16]:
y_pred_train = pd.DataFrame(y_pred_train, columns=['Predicted_y'])

diff_pred_true_train = pd.concat([y_pred_train, train_y], axis=1)

diff_pred_true_train['Difference'] = diff_pred_true_train.Predicted_y - diff_pred_true_train.Publication_year
    

print(diff_pred_true_train.head(3))


   Predicted_y  Publication_year  Difference
0  1786.259536              1741   45.259536
1  1750.030794              1691   59.030794
2  1770.757624              1665  105.757624


In [17]:
diff_pred_true_train.describe()

,Predicted_y,Publication_year,Difference
count,899.000000,899.000000,8.990000e+02
mean,1788.259177,1788.259177,-4.426073e-14
std,61.741568,77.929074,4.756380e+01
min,1549.902463,1598.000000,-1.258733e+02
25%,1762.839429,1739.500000,-2.805162e+01
50%,1782.950146,1796.000000,2.362674e-02
75%,1824.280243,1855.000000,2.514667e+01
max,1962.413722,1913.000000,1.671227e+02


This table shows that the mean of the years is the same for the predicted and the true year. The maximum and minimum of the model's prediction is lower and higher than in the true labels, so the model thinks that the range between the earliest and the latest publication year is larger than shown in the train set.

In [18]:
y_pred_val = pd.DataFrame(y_pred_val, columns=['Predicted_y'])

diff_pred_true_val = pd.concat([y_pred_val, val_y], axis=1)

diff_pred_true_val['Difference'] = diff_pred_true_val.Predicted_y - diff_pred_true_val.Publication_year

print(diff_pred_true_val.head(3))


   Predicted_y  Publication_year  Difference
0  1818.768364              1897  -78.231636
1  1717.750097              1701   16.750097
2  1517.937050              1663 -145.062950


In [19]:
diff_pred_true_val.describe()

,Predicted_y,Publication_year,Difference
count,225.000000,225.000000,225.000000
mean,1789.651089,1791.315556,-1.664467
std,70.223421,74.822785,56.692355
min,1482.561566,1603.000000,-164.438434
25%,1763.628141,1750.000000,-35.216816
50%,1784.658281,1804.000000,-5.313875
75%,1822.392752,1843.000000,28.570888
max,2025.898784,1913.000000,255.349277


The true mean of the publication year in the validation set is three years higher than in the train set. The model adapts slightly by adding one year to the mean of the predictions over the validation set (compared to the train set).

Surprisingly, the model dates the oldest text from the validation set back to 1462, when in fact, the oldest text was written in 1603. The youngest text in the validation set, according to the model, was written in 2014, the true year of the youngest text is 1913. This means that the range of the prediction is about 350 years larger than it should be.

The mean difference between the predicted and the true year is -2, meaning that the predicted year is generally two years lower than the true label.

in the first quartile, the models prediction is about 37 years to low, in the third quartile, the prediction is about 256 years to high. It seems that the model generally tends to predict a higher publication year than the true year. Given the mean (which is actually quite decent), the main problem might be some heavy outliers in the model's prediction.

In [21]:
y_pred_test = pd.DataFrame(y_pred_test, columns=['Predicted_y'])

diff_pred_true_test = pd.concat([y_pred_test, test_y], axis=1)

diff_pred_true_test['Difference'] = diff_pred_true_test.Predicted_y - diff_pred_true_test.Publication_year

print(diff_pred_true_test.head(3))

   Predicted_y  Publication_year  Difference
0  1872.240933              1861   11.240933
1  1813.594824              1801   12.594824
2  1751.301201              1790  -38.698799


In [18]:
print(diff_pred_true_val.nsmallest(10,'Difference'))

     Predicted_y  Publication_year  Difference
143  1482.561566              1647 -164.438434
183  1531.163409              1679 -147.836591
2    1517.937050              1663 -145.062950
170  1763.628141              1895 -131.371859
54   1775.478628              1897 -121.521372
188  1779.429106              1893 -113.570894
159  1783.201390              1890 -106.798610
50   1784.179079              1889 -104.820921
47   1801.314523              1898  -96.685477
116  1820.343756              1913  -92.656244


eli5 instance 119 and 142: The model predicts 2012 and 2014 (true: 1765 and 1895), probably because it overestimates the influence of the word "dem". In the instance of 2012, "dem" has a weight of +348, whereas for the one with 2014, it is weighted with +71. 

"die" seems also to be a word that misleads the classifier to think that a text is younger than it really is. For the text of 2012, "die" has a weight of +185, for the example that was predicted with 2014, the weight is +65

In [22]:
diff_pred_true_val.to_csv('/Volumes/Korpora/DTA_Reg4_Labels_val.csv',sep=';')

In [23]:
diff_pred_true_train.to_csv('/Volumes/Korpora/DTA_Reg4_Labels_train.csv',sep=';')

In [25]:
diff_pred_true_test.to_csv('/Volumes/Korpora/DTA_Reg4_Labels_test.csv',sep=';')

In [19]:
#eli5.explain_prediction(reg_4['ridge_reg'],val_x[143],vec=reg_4['unigram_vectorizer'], feature_names=features_selected, )

In [20]:
expl = eli5.explain_prediction_df(reg_4['ridge_reg'],val_x[0],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)



In [21]:
len(expl)

213

In [22]:
expl.nsmallest(10,'value')

,target,feature,weight,value
0,y,<BIAS>,1769.724984,1.0
111,y,'welchem',0.112434,1.0
113,y,'gleichen',0.080908,1.0
100,y,'findet',0.240282,2.0
116,y,'welches',0.016883,2.0
117,y,'nen',0.014924,2.0
106,y,'bleibt',0.174982,3.0
127,y,'zeiten',-0.310498,3.0
83,y,'liegen',0.608372,4.0
112,y,'dahin',0.083053,4.0


In [23]:
eli5.explain_weights_df(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

,target,feature,weight
0,y,<BIAS>,1769.724984
1,y,'keine',0.282159
2,y,'weiter',0.260575
3,y,'ja',0.216977
4,y,'that',0.181377
...,...,...,...
211,y,'gen',-0.176596
212,y,'bald',-0.179685
213,y,'ins',-0.221891
214,y,'weit',-0.229857


In [26]:
val_details = collect_predictions(val_x, reg_4['ridge_reg'],reg_4['unigram_vectorizer'],features_selected, reg_4)



In [91]:
len(val_details)

46786

In [27]:
train_details = collect_predictions(train_x, reg_4['ridge_reg'],reg_4['unigram_vectorizer'],features_selected, reg_4)

In [28]:
test_details = collect_predictions(test_x, reg_4['ridge_reg'],reg_4['unigram_vectorizer'],features_selected, reg_4)

In [29]:
train_details.to_csv('/Volumes/Korpora/Exp2_Reg4_Train_results_DTA.csv',sep=';')
val_details.to_csv('/Volumes/Korpora/Exp2_Reg4_Val_results_DTA.csv', sep=';')
test_details.to_csv('/Volumes/Korpora/Exp2_Reg4_Test_results_DTA.csv', sep=';')

**REG4 + CLMET**

In [55]:
train_clmet_full = pd.read_csv('/Volumes/Korpora/Train/CLMET_train_tokenized.csv', sep=';')
val_clmet_full = pd.read_csv('/Volumes/Korpora/Val/CLMET_val_tokenized.csv', sep=';')
test_clmet_full = pd.read_csv('/Volumes/Korpora/Test/CLMET_test_tokenized.csv', sep=';')

In [56]:
print('Length train set: ',len(train_clmet_full))
print('Length validation set: ', len(val_clmet_full))
print('Length test set: ', len(test_clmet_full))

Length train set:  212
Length validation set:  54
Length test set:  67


In [67]:
#drop rows with invalid data types
train_clmet_full = train_clmet_full[train_clmet_full.Year.str.len()== 4]
val_clmet_full = val_clmet_full[val_clmet_full.Year.str.len()== 4]
test_clmet_full = test_clmet_full[test_clmet_full.Year.str.len()== 4]

In [68]:
print('Length train set: ',len(train_clmet_full))
print('Length validation set: ', len(val_clmet_full))
print('Length test set: ', len(test_clmet_full))

Length train set:  186
Length validation set:  47
Length test set:  60


In [69]:
train_x = train_clmet_full['Text']
train_y = train_clmet_full['Year'].astype(int)

val_x = val_clmet_full['Text']
val_y = val_clmet_full['Year'].astype(int)

test_x = test_clmet_full['Text']
test_y = test_clmet_full['Year'].astype(int)

90% von 186 = 167.4

In [70]:
reg_4 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 167)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [71]:
reg_4.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('unigram_vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=167,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_word at 0x1d9359c20>,
                                 vocabulary=None)),
                ('feature_selector',
                 SelectKBest(k='all',
                             score_func=<function f_regression at 0x12071cdd0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr

In [72]:
y_pred_train = reg_4.predict(train_x)
mean_squared_error(train_y, y_pred_train)

0.0032282565763734907

In [73]:
y_pred_val = reg_4.predict(val_x)
mean_squared_error(val_y, y_pred_val)

28696.790663825832

In [74]:
features = reg_4['feature_selector'].get_support()
feature_names = reg_4['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.explain_weights(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

Explanation(estimator="Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,\n      normalize=False, random_state=None, solver='auto', tol=0.001)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1845.241183301348, std=None, value=None), FeatureWeight(feature=" 'well'", weight=0.7932533575609532, std=None, value=None), FeatureWeight(feature=" 'believe'", weight=0.5553470814343288, std=None, value=None), FeatureWeight(feature=" 'understand'", weight=0.5067271664956602, std=None, value=None), FeatureWeight(feature=" 'place'", weight=0.43223521147661753, std=None, value=None), FeatureWeight(feature=" 'times'", weight=0.3749313075045127, std=None, value=None), FeatureWeight(feature=" 'given'", weight=0.3686209540580662, std=None, value=None), FeatureWeight(feature=" 'another'", weight=0.3677556164614103, std=None, value=None)], neg=[FeatureWeight(feature=" 'here'", weight=-0.6118710618637919, std=None, value=None), FeatureWeight(feature=" 'air'", weight=-0.5363199562704768, std=None, value=None), FeatureWeight(feature=" 'family'", weight=-0.5091213314847125, std=None, value=None), FeatureWeight(feature=" 'shall'", weight=-0.497083218632966, std=None, value=None), FeatureWeight(feature=" 'heart'", weight=-0.48527491925067245, std=None, value=None), FeatureWeight(feature=" 'an'", weight=-0.45654096167390223, std=None, value=None), FeatureWeight(feature=" 'went'", weight=-0.4438697765496704, std=None, value=None), FeatureWeight(feature=" 'great'", weight=-0.417272462679933, std=None, value=None), FeatureWeight(feature=" 'cause'", weight=-0.38559367151203205, std=None, value=None), FeatureWeight(feature=" 'use'", weight=-0.3794881842952177, std=None, value=None), FeatureWeight(feature=" 'having'", weight=-0.37687363848288846, std=None, value=None), FeatureWeight(feature=" 'told'", weight=-0.3724742980133045, std=None, value=None)], pos_remaining=217, neg_remaining=263), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [75]:
len(features)

499

Restrict features on the same number as the model chooses for the DTA

In [78]:
reg_4 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 167)),
                    ('feature_selector', SelectKBest(f_regression, k=215)),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [79]:
reg_4.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('unigram_vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=167,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_word at 0x1d9359c20>,
                                 vocabulary=None)),
                ('feature_selector',
                 SelectKBest(k=215,
                             score_func=<function f_regression at 0x12071cdd0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True

In [80]:
y_pred_train = reg_4.predict(train_x)
mean_squared_error(train_y, y_pred_train)

0.013660963035626862

In [81]:
y_pred_val = reg_4.predict(val_x)
mean_squared_error(val_y, y_pred_val)

46532.69500113356

In [82]:
features = reg_4['feature_selector'].get_support()
feature_names = reg_4['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.explain_weights(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

Explanation(estimator="Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,\n      normalize=False, random_state=None, solver='auto', tol=0.001)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1831.5779899900551, std=None, value=None), FeatureWeight(feature=" 'understand'", weight=5.394649735828971, std=None, value=None), FeatureWeight(feature=" 'met'", weight=2.7587614668556704, std=None, value=None), FeatureWeight(feature=" 'lie'", weight=2.7433084361602944, std=None, value=None), FeatureWeight(feature=" 'remember'", weight=2.54367709119154, std=None, value=None), FeatureWeight(feature=" 'appear'", weight=2.534694072036934, std=None, value=None), FeatureWeight(feature=" 'got'", weight=2.1554588634890806, std=None, value=None), FeatureWeight(feature=" 'danger'", weight=2.0526740637627996, std=None, value=None), FeatureWeight(feature=" 'hands'", weight=1.9004026797438665, std=None, value=None)], neg=[FeatureWeight(feature=" 'sound'", weight=-3.077244019749402, std=None, value=None), FeatureWeight(feature=" 'wrong'", weight=-2.7207008646535313, std=None, value=None), FeatureWeight(feature=" 'fit'", weight=-2.6696896173856683, std=None, value=None), FeatureWeight(feature=" 'interest'", weight=-2.4341035222302896, std=None, value=None), FeatureWeight(feature=" 'allow'", weight=-2.2277614283025553, std=None, value=None), FeatureWeight(feature=" 'cause'", weight=-2.1771458670655837, std=None, value=None), FeatureWeight(feature=" 'care'", weight=-2.1222297791112608, std=None, value=None), FeatureWeight(feature=" 'consider'", weight=-2.06032568655359, std=None, value=None), FeatureWeight(feature=" 'hold'", weight=-2.047716560949669, std=None, value=None), FeatureWeight(feature=" 'besides'", weight=-1.9859712697584366, std=None, value=None), FeatureWeight(feature=" 'written'", weight=-1.945164164931982, std=None, value=None)], pos_remaining=87, neg_remaining=109), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

Ratio between features and number of documents from experiments with the DTA:
899 to 215 = 167 to 40

In [83]:
reg_4 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 167)),
                    ('feature_selector', SelectKBest(f_regression, k=40)),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [84]:
reg_4.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('unigram_vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=167,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_word at 0x1d9359c20>,
                                 vocabulary=None)),
                ('feature_selector',
                 SelectKBest(k=40,
                             score_func=<function f_regression at 0x12071cdd0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,

In [85]:
y_pred_train = reg_4.predict(train_x)
mean_squared_error(train_y, y_pred_train)

1346.2602790843573

In [86]:
y_pred_val = reg_4.predict(val_x)
mean_squared_error(val_y, y_pred_val)

2727.4587338608444

In [87]:
features = reg_4['feature_selector'].get_support()
feature_names = reg_4['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.explain_weights(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

Weight?,Feature
+1850.135,<BIAS>
+1.109,'need'
+0.793,'understand'
+0.656,'question'
+0.596,'coming'
+0.491,'occasion'
+0.320,'behind'
+0.307,'right'
+0.256,'fear'
+0.225,'forward'
